## ENACT Demo Notebook - Human Colorectal Cancer

This notebook provides a demo for running ENACT on the Human Colorectal Cancer sample provided on 10X Genomics' website.

### Download VisiumHD data from the 10X Genomics website

Whole slide image: full resolution tissue image

In [ ]:
!curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.0.0/Visium_HD_Human_Colon_Cancer/Visium_HD_Human_Colon_Cancer_tissue_image.btf

Visium HD output file. The transcript counts are provided in a .tar.gz file that needs to be extracted:

In [ ]:
!curl -O https://cf.10xgenomics.com/samples/spatial-exp/3.0.0/Visium_HD_Human_Colon_Cancer/Visium_HD_Human_Colon_Cancer_binned_outputs.tar.gz
!tar -xvzf Visium_HD_Human_Colon_Cancer_binned_outputs.tar.gz

Locate the following two files from the extracted outputs file. These are the files we will use later as input to ENACT.


In [ ]:
.
└── binned_outputs/
    └── square_002um/
        ├── filtered_feature_bc_matrix.h5   <---- Transcript counts file (2um resolution)
        └── spatial/
            └── tissue_positions.parquet    <---- Bin locations relative to the full resolution image


### Install ENACT
This will install the ENACT package and its dependencies.


In [ ]:
!pip install enact-SO

### Access and update the `configs.yaml` file

To run the ENACT pipeline, you will need a configuration file that specifies all the required settings. You can download the template configuration file from the GitHub repository.

Refer to [Defining ENACT Configurations](https://github.com/Sanofi-OneAI/oneai-dda-spatialtr-enact/tree/release/ospo-new?tab=readme-ov-file#defining-enact-configurations) for a full list of parameters to configure.

#### Step 1
Download the `configs.yaml` template from the `config` folder of [this repository](https://github.com/Sanofi-OneAI/oneai-dda-spatialtr-enact), and save it in your working directory.

#### Step 2
Edit the input file locations in `configs.yaml` to the downloaded Visium HD files' location.

```yaml
analysis_name: "demo-colon"                           
cache_dir: "enact_output"                   
paths:
    wsi_path: "Visium_HD_Human_Colon_Cancer_tissue_image.btf"      
    visiumhd_h5_path: "binned_outputs/square_002um/filtered_feature_bc_matrix.h5"         
    tissue_positions_path: "binned_outputs/square_002um/spatial/tissue_positions.parquet"      
```

#### Step 3
Next, we set all the steps in the `configs.yaml` file to `True`, in order to run the whole ENACT pipeline later

```yaml
steps:
        segmentation: True                                   
        bin_to_geodataframes: True                             
        bin_to_cell_assignment: True                           
        cell_type_annotation: True         
```

#### Step 4
Lastly, choose the `bin_to_cell_method` and `cell_annotation_method` we want to run with. In this demo, we will go with `"weighted_by_area"`, and `"celltypist"`.

To run Celltypist as our cell annotation method, we also need to input the `cell_typist_model` parameter based on the type of sample we use.

```yaml
 params:
      bin_to_cell_method: "weighted_by_area"                
      cell_annotation_method: "celltypist"                     
      cell_typist_model: "Human_Colorectal_Cancer.pkl"         
      seg_method: "stardist"                                    
      patch_size: 4000                                         
      use_hvg: True                                            
      n_hvg: 1000                                              
```

### Run ENACT

Running ENACT on the whole sample image will take around 40 minutes. Output of the pipeline will be stored in the `"enact_output"` directory

In [ ]:
from enact.pipeline import ENACT
import yaml

configs_path = "config/configs.yaml" # Change this to the location of the configs.yaml file that you just edited
with open(configs_path, "r") as stream:
    configs = yaml.safe_load(stream)

so_hd = ENACT(configs_dict=configs)
so_hd.run_enact()

New! Alternatively, users can specify ENACT configurations directly in the class constructor with the following *minimum* configurations. Refer to Readme for full list of ENACT parameters.

In [ ]:
from enact.pipeline import ENACT

# Running ENACT with `weighted-by-area` bin-to-cell assignment, and `celltypist` for cell type annotation
so_hd = ENACT(
    cache_dir="/home/oneai/test_cache",
    wsi_path="Visium_HD_Human_Colon_Cancer_tissue_image.btf",
    visiumhd_h5_path="binned_outputs/square_002um/filtered_feature_bc_matrix.h5",
    tissue_positions_path="binned_outputs/square_002um/spatial/tissue_positions.parquet",
    analysis_name="demo-colon", #optional
    bin_to_cell_method="weighted_by_area", #optional    
    cell_annotation_method="celltypist", #optional                
    cell_typist_model="Human_Colorectal_Cancer.pkl" #optional  
)
so_hd.run_enact()

Example: Only running the cell segmentation step and disabling all the other steps

In [ ]:
from enact.pipeline import ENACT

so_hd = ENACT(
    cache_dir="/home/oneai/test_cache",
    wsi_path="Visium_HD_Human_Colon_Cancer_tissue_image.btf",
    visiumhd_h5_path="binned_outputs/square_002um/filtered_feature_bc_matrix.h5",
    tissue_positions_path="binned_outputs/square_002um/spatial/tissue_positions.parquet",
    analysis_name="demo-colon",
    bin_to_cell_method="weighted_by_area",    
    cell_annotation_method="celltypist",                
    cell_typist_model="Human_Colorectal_Cancer.pkl",
    segmentation=True,
    bin_to_geodataframes=False,
    bin_to_cell_assignment=False,
    cell_type_annotation=False
)
so_hd.run_enact()

Example: Running ENACT with `naive` bin-to-cell assignment and `cellassign` for cell type annotation

In [ ]:
from enact.pipeline import ENACT

so_hd = ENACT(
    cache_dir="/home/oneai/test_cache",
    wsi_path="Visium_HD_Human_Colon_Cancer_tissue_image.btf",
    visiumhd_h5_path="binned_outputs/square_002um/filtered_feature_bc_matrix.h5",
    tissue_positions_path="binned_outputs/square_002um/spatial/tissue_positions.parquet",
    analysis_name="demo-colon",
    bin_to_cell_method="naive",    
    cell_annotation_method="cellassign"
)
so_hd.run_enact()